In [ ]:
# Import necessary libraries
import pandas as pd
import graph_tool.all as gt
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import linregress
from collections import Counter
from PIL import Image
import matplotlib.patches as mpatches
import io


# Load node and edge data
try:
    nodes2 = pd.read_csv("./edges_longcovid_processed_cut_weight5.edges_node_info_merge.csv", sep=";")
    edges = pd.read_csv('./edges_membership_layout_w2-w20/edges_longcovid_processed_cut_weight5.edges', sep=" ", header=None)
except FileNotFoundError as e:
    print(f"Error: {e}")
    raise

# Drop rows with missing ReliabilityIndex
nodes2 = nodes2.dropna(subset=['ReliabilityIndex'])

# Rename edge columns for clarity
edges.rename(columns={0: 'UserFrom', 1: 'UserTo', 2: 'weights'}, inplace=True)

# Filter nodes that are in edges
nodes_ = nodes2[nodes2['User'].isin(edges['UserTo']) | nodes2['User'].isin(edges['UserFrom'])]
nodes_['id'] = range(1, 1 + len(nodes_))

# Merge nodes data with edges to create source and target mappings
edges_ = pd.merge(edges, nodes_[['User', 'id']], left_on='UserFrom', right_on='User')
edges_.rename(columns={'id': 'source'}, inplace=True)
edges_ = pd.merge(edges_, nodes_[['User', 'id']], left_on='UserTo', right_on='User')
edges_.rename(columns={'id': 'target'}, inplace=True)

# Remove unnecessary columns
edges_.drop(columns=['User_x', 'User_y'], inplace=True)


# Create node and edge mappings
node_map = {row.Index: row.User for row in nodes_.itertuples()}
node_map_inv = {v: k for k, v in node_map.items()}
edge_map = {row.Index: (row.source, row.target) for row in edges_.itertuples()}
edge_map_inv = {v: k for k, v in edge_map.items()}

# Assign weights to edges
n_weights = {(edge.source, edge.target): edge.weights for edge in edges_.itertuples()}

# Initialize graph
g = gt.Graph(directed=False)
g.add_vertex(len(node_map))

# Add edges to the graph
for source, target in n_weights:
    g.add_edge(g.vertex(source), g.vertex(target))

# Compute layout for visualization
pos = gt.sfdp_layout(g)

# Assign vertex properties
vertex_name = g.new_vertex_property("string", vals=nodes_['User'].values)
n_weights_pm = g.new_edge_property("float", vals=list(n_weights.values()))
g.edge_properties["n_weights"] = n_weights_pm

# Minimize nested blockmodel
state = gt.minimize_nested_blockmodel_dl(g, state_args=dict(deg_corr=True))
state.print_summary()

# Save initial entropy
initial_entropy = state.entropy()

# Perform MCMC sweeps for model refinement
for _ in range(1000):
    state.multiflip_mcmc_sweep(beta=np.inf, niter=10)

# Calculate improvement in entropy
final_entropy = state.entropy()
print(f"Improvement in entropy: {final_entropy - initial_entropy}")

# Assign layout and vertex names to graph properties
g.vertex_properties["pos"] = pos
g.vertex_properties["vertex_name"] = vertex_name

# Visualization of community structure
cmap = plt.get_cmap('rainbow')
min_nodes_in_community = 100

# Loop through hierarchical levels
for level in range(len(state.levels) - 1):
    block_projection = state.project_partition(level, 0)
    print(f"Level: {level}")
    
    # Create a buffer to hold the image
    buf = io.BytesIO()
    
    # Draw the graph and save to buffer
    gt.graph_draw(
        g,
        pos=g.vp.pos,
        output_size=(3000, 3000),
        edge_pen_width=2.0,
        edge_color=[1, 1, 1, 0.1],
        vertex_fill_color=block_projection,
        vcmap=(cmap, 0.5),
        vertex_color=[1, 1, 1, 0.2],
        bg_color=[1, 1, 1, 1],
        output=buf,
        fmt='png'
    )
    
    # Move buffer to start
    buf.seek(0)
    
    # Open image and display
    image = Image.open(buf)
    fig, ax = plt.subplots(figsize=(12, 12), dpi=900)
    ax.imshow(np.array(image))
    ax.axis('off')
    
    # Compute community sizes
    community_sizes = Counter(block_projection.a)
    sorted_communities = sorted(community_sizes.items(), key=lambda x: x[1], reverse=True)
    largest_communities = sorted_communities[:10]
    norm = plt.Normalize(vmin=min(community_sizes.values()), vmax=max(community_sizes.values()))
    
    # legend for largest communities
    legend_patches = [mpatches.Patch(color=cmap(norm(comm)), label=f'Community {comm} ({size} nodes)') for comm, size in largest_communities]
    plt.legend(handles=legend_patches, loc='lower right', bbox_to_anchor=(1.1, 0), title="Top 10 Largest Communities")
    
    # Save the final image
    plt.savefig(f"{level}_level.png", dpi=1000, bbox_inches='tight')
    
    # Close buffer
    buf.close()
